# Setup e Checagens

Notebook para a configuração inicial de ambiente para execução de tarefas com Apache Spark. Etapas:
- Carrega as configurações do projeto;
- Define as seeds para garantir reprodutibilidade dos experimentos;
- Inicializa uma sessão Spark personalizada conforme os parâmetros definidos. 

Ao final, exibe uma amostra de dados gerados pelo Spark e encerra a sessão.

In [9]:
import os, sys, subprocess
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"

IN_COLAB = "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

if IN_COLAB:
    PROJECT_DIR = Path("/content") / REPO_NAME
    if not PROJECT_DIR.exists():
        print(f"Clonando https://github.com/{GITHUB_USER}/{REPO_NAME}.git ...")
        subprocess.check_call([
            "git", "clone", f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git", str(PROJECT_DIR)
        ])
    os.chdir(PROJECT_DIR)
else:
    PROJECT_DIR = Path.cwd()
    if not (PROJECT_DIR / "requirements.txt").exists():
        if PROJECT_DIR.name == "notebooks" and (PROJECT_DIR.parent / "requirements.txt").exists():
            os.chdir(PROJECT_DIR.parent)
            PROJECT_DIR = PROJECT_DIR.parent
        else:
            raise RuntimeError("Abra o Jupyter na RAIZ do projeto (onde está requirements.txt).")

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

if IN_COLAB:
    print("Instalando dependências (requirements.txt)...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])

print("Baixando/extraindo dados (scripts/download_data.py)...")
subprocess.check_call([sys.executable, "scripts/download_data.py"])
print("✔️ Dados prontos em data/raw/")

Baixando/extraindo dados (scripts/download_data.py)...
✔️ Dados prontos em data/raw/


In [10]:
from src.utils import load_settings, set_seeds, get_spark, stop_spark

s = load_settings()                   # lê config (settings.yaml)
set_seeds(s.runtime.seed)             # fixa seeds para reprodutibilidade
spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions,
)
print("Spark OK:", spark.version)
spark.range(5).show()
stop_spark(spark)


Spark OK: 3.5.1
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

